In [1]:
import pandas as pd
import time
from datetime import datetime, timedelta
from pytz import timezone
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from multiprocessing import Process, Queue

In [23]:
pd.set_option('display.max_columns', None)

In [2]:
chats_beta = pd.read_csv("../data/train/chats_train.csv")

In [3]:
chats_beta.head()

,chat_id,chat_title,emoji_list,sender_first_name,sender_id,sender_is_bot,sender_last_name,sender_username,text,timestamp
0,1129149026,Pareto Network - Discussion,[],Bezant,526106862,False,None,None,Admin disputes in private thank you.,1524284160
1,1129149026,Pareto Network - Discussion,[],Sheriff,538526216,False,None,None,I see Binance add coins all the time on the Cr...,1524284160
2,1146170349,Binance English,[],DaySandBox,400753764,True,None,daysandbox_bot,Removed msg from __Megzon Kadrija__. Reason: n...,1524285614
3,1146170349,Binance English,[],Kamalendu,435609627,False,Pal,None,Bc xvg dead,1524285372
4,1146170349,Binance English,[':thumbsup:'],Abdelbari,154857846,False,K,None,"Yes, youre right 👍",1524286342


In [4]:
chats_beta.shape

(2888606, 10)

In [5]:
chats_beta.isnull().sum()

chat_id                   0
chat_title               19
emoji_list                0
sender_first_name        48
sender_id                 0
sender_is_bot             0
sender_last_name        246
sender_username        2494
text                 157523
timestamp                 0
dtype: int64

In [7]:
chats_beta.groupby("chat_id")["chat_title"].count().nlargest(20)

chat_id
1146170349    507310
1128794996    442539
1012147388    215515
1009909164    183064
1122948260    136857
1232812654    123701
1224911847    121980
1224810332    105134
1127724611     80111
1139159476     79024
1112237776     72711
1198729259     71456
1072359623     65196
1119581721     62424
1134260225     57736
1249433091     52903
1122422145     50239
1123270218     48135
1181367025     42923
1068163054     35105
Name: chat_title, dtype: int64

In [8]:
len(chats_beta["chat_id"].unique())

63

In [9]:
len(chats_beta["sender_id"].unique())

203132

In [10]:
chats_beta.groupby("sender_id")["chat_title"].count().nlargest(20)

sender_id
153606345    14231
387537131    11548
496104690    10909
524958633    10143
172018489     9993
81939250      8444
176365905     8444
355285798     8158
216901420     7204
402367678     6661
488950231     6496
463297575     5980
498224021     5923
520650165     4846
40316752      4726
434441956     4631
429765676     4617
372936297     4549
494301030     4549
188112857     4509
Name: chat_title, dtype: int64

In [ ]:
def markDataFrameByChunks(df, num, col):
    ordNums = np.array(sorted(df[col].unique(),reverse=False))
    ordRange = range(0, len(ordNums))
    chunksDF = pd.DataFrame({'colName': ordNums, 'theirNum': ordRange}, index = ordRange)
    chunksDF['chunk'] = pd.cut(chunksDF['theirNum'], num)
    return chunksDF

In [29]:
news_beta = pd.read_csv("../data/sample/news_beta.csv")

In [31]:
news_beta.head(50)

,link,title,publication_date,content,date,site,section
0,https://www.reddit.com/r/CryptoCurrency/commen...,Introduction of ICONest: ICON’s Token Launch P...,2018-04-30 10:49:31,Introduction of ICONest: ICON’s Token Launch P...,2018-04-30 13:21:07.376+02,www.reddit.com,CryptoCurrency
1,https://www.reddit.com/r/CryptoCurrency/commen...,APEX Network Korean Community Launch Announcement,2018-03-04 10:49:55,APEX Network Korean Community Launch Announcement,2018-03-04 13:00:18.037+01,www.reddit.com,CryptoCurrency
2,https://cointelegraph.com/news/april-fools-day...,April Fool’s Day Rundown - Nerding Out On Cryp...,2018-04-01 14:14:00,Every year on April 1 - April Fool’s Day - mem...,2018-04-01 19:00:17.171+02,cointelegraph.com,NaN
3,https://www.reddit.com/r/btc/comments/8i2fus/y...,"You can already do microtransactions on-chain,...",2018-05-09 02:15:34,"You can already do microtransactions on-chain,...",2018-05-09 06:14:24.913+02,www.reddit.com,btc
4,https://www.reddit.com/r/CryptoCurrency/commen...,"Decentralised exchange ""next.exchange"" start's...",2018-03-01 23:46:15,"Decentralised exchange ""next.exchange"" start's...",2018-03-02 01:17:57.228+01,www.reddit.com,CryptoCurrency
5,https://www.coindesk.com/new-york-power-provid...,New York Power Providers Cleared to Hike Rates...,2018-03-16 13:00:40,Cryptocurrency mining firms in upstate New Yor...,2018-03-16 14:01:07.478+01,www.coindesk.com,NaN
6,https://www.reddit.com/r/Bitcoin/comments/8gza...,Vaultoro Becomes The First Exchange To Impleme...,2018-05-04 13:34:39,Vaultoro Becomes The First Exchange To Impleme...,2018-05-04 17:00:12.539+02,www.reddit.com,Bitcoin
7,https://www.reddit.com/r/CryptoCurrency/commen...,"If you invested in the delay scam known as ""Xt...",2018-04-10 15:13:59,"If you invested in the delay scam known as ""Xt...",2018-04-10 18:00:16.279+02,www.reddit.com,CryptoCurrency
8,https://steemit.com/bitcoin/@plapuma/a-new-vie...,A New View On The Cryptoverse: #introduceyours...,2018-03-06 12:33:09,![bitcoin-3146330_960_720.jpg](https://steemit...,2018-03-06 18:00:13.088+01,steemit.com,NaN
9,https://www.coindesk.com/the-crown-prince-of-l...,The Crown Prince of Liechtenstein Wants to Inv...,2018-03-16 00:50:28,"There may be a literal ""crypto-king"" one day i...",2018-03-16 02:00:14.644+01,www.coindesk.com,NaN


In [14]:
onchain_beta_EOS = pd.read_csv("../data/sample/onchain_beta_EOS.csv")

In [15]:
onchain_beta_EOS.head()

,from,to,value,timestamp
0,e4582,c08a9,3.081605e+22,1524723037
1,6173,6331c,2.000000e+18,1527592584
2,3d26,67dfe,2.373771e+20,1525413428
3,bc084,8a22c,1.000000e+19,1525133308
4,a8abe,5c293,2.160000e+22,1525410762


In [32]:
prices_beta = pd.read_csv("../data/sample/prices_beta.csv")

In [33]:
prices_beta.tail()

,ticker,datetime,priceBtc,priceUsd,volume
95,NULS,2018-04-20T12:00:00Z,0.000309,2.629457,12895600.0
96,CPY,2018-04-30T18:00:00Z,0.000022,0.202940,10371.0
97,CVC,2018-03-05T02:00:00Z,0.000031,0.358627,3897670.0
98,NXS,2018-04-29T06:00:00Z,0.000259,2.445498,3026580.0
99,LUN,2018-03-28T20:00:00Z,0.001213,9.588604,2808830.0


In [20]:
prices_beta.groupby("ticker")["priceBtc"].count().nlargest()

ticker
1ST    2
BRD    2
CPY    2
ELA    2
ERA    2
Name: priceBtc, dtype: int64

find wallets, that start rises and falls

(1, 5)